In [42]:
import os
print(os.getcwd())
import sys
import os
import re
import compare_csv
import openai as OpenAI
from typing import List, Tuple, Dict
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from rapidfuzz.distance.Levenshtein import distance
from rapidfuzz import fuzz
import re

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


current_dir = os.path.dirname(os.path.abspath("/d/GithubRepos/PIMCO-Text2SQL"))
din_modules_path = os.path.join(current_dir, 'chatgpt_api')
sys.path.append(din_modules_path)

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
if not client.api_key:
    raise ValueError("OpenAI API key not configured")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/virounikamina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/virounikamina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/virounikamina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/virounikamina/Desktop/PIMCO-Text2SQL/env/lib/python3.11/site-packages/certifi/cacert.pem'


/Users/virounikamina/Desktop/PIMCO-Text2SQL


In [ ]:
def explore_keys():
    """Explore potential primary and foreign keys in the database"""
    import sqlite3
    
    # Connect to database
    conn = sqlite3.connect('sqlite/nport.db')
    cursor = conn.cursor()

    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    print("Database Key Analysis:")
    print("-" * 80)

    # Analyze each table
    for table in tables:
        table_name = table[0]
        print(f"\nTable: {table_name}")

        # Get column info
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        
        # Get sample count for potential key columns
        for col in columns:
            col_name = col[1]
            # Check if column name contains potential key indicators
            if any(key_term in col_name.lower() for key_term in ['_id', 'accession', 'number']):
                cursor.execute(f"""
                    SELECT COUNT(*) total_rows, 
                           COUNT(DISTINCT {col_name}) unique_values 
                    FROM {table_name}
                    WHERE {col_name} IS NOT NULL
                """)
                stats = cursor.fetchone()
                print(f"Column: {col_name}")
                print(f"Total rows: {stats[0]}")
                print(f"Unique values: {stats[1]}")
                
                # If unique values equals total rows, likely a key
                if stats[0] == stats[1] and stats[0] > 0:
                    print(">>> Potential PRIMARY KEY <<<")

        # Look for foreign key relationships
        for col in columns:
            col_name = col[1]
            if col_name == 'ACCESSION_NUMBER':
                cursor.execute(f"""
                    SELECT COUNT(*) FROM {table_name} t1
                    WHERE EXISTS (
                        SELECT 1 FROM FUND_REPORTED_INFO t2 
                        WHERE t1.ACCESSION_NUMBER = t2.ACCESSION_NUMBER
                    )
                """)
                match_count = cursor.fetchone()[0]
                if match_count > 0:
                    print(f"Foreign Key: {table_name}.ACCESSION_NUMBER -> FUND_REPORTED_INFO.ACCESSION_NUMBER")
            
            elif col_name == 'HOLDING_ID':
                cursor.execute(f"""
                    SELECT COUNT(*) FROM {table_name} t1
                    WHERE EXISTS (
                        SELECT 1 FROM FUND_REPORTED_HOLDING t2 
                        WHERE t1.HOLDING_ID = t2.HOLDING_ID
                    )
                """)
                match_count = cursor.fetchone()[0]
                if match_count > 0:
                    print(f"Foreign Key: {table_name}.HOLDING_ID -> FUND_REPORTED_HOLDING.HOLDING_ID")

    conn.close()

# Run the analysis
explore_keys()

In [ ]:
text = """
1. "List the top 5 registrants by total net assets, including their CIK and country."
   SQL: 
   WITH FundAssets AS (
       SELECT R.CIK, R.REGISTRANT_NAME, R.COUNTRY, F.NET_ASSETS
       FROM REGISTRANT R
       JOIN FUND_REPORTED_INFO F ON R.ACCESSION_NUMBER = F.ACCESSION_NUMBER
   )
   SELECT CIK, REGISTRANT_NAME, COUNTRY, NET_ASSETS
   FROM FundAssets
   ORDER BY NET_ASSETS DESC
   LIMIT 5;

2. "Find all holdings with a fair value level of Level 1 and their corresponding fund names."
   SQL: 
   WITH HoldingsCTE AS (
       SELECT H.HOLDING_ID, H.ISSUER_NAME, H.FAIR_VALUE_LEVEL, F.SERIES_NAME
       FROM FUND_REPORTED_HOLDING H
       JOIN FUND_REPORTED_INFO F ON H.ACCESSION_NUMBER = F.ACCESSION_NUMBER
       WHERE H.FAIR_VALUE_LEVEL = 'Level 1'
   )
   SELECT HOLDING_ID, ISSUER_NAME, SERIES_NAME
   FROM HoldingsCTE;

3. "Calculate the total collateral amount for repurchase agreements grouped by counterparty."
   SQL: 
   WITH CollateralCTE AS (
    SELECT RCP.NAME AS Counterparty_Name, SUM(RC.COLLATERAL_AMOUNT) AS Total_Collateral
    FROM REPURCHASE_COLLATERAL RC
    JOIN REPURCHASE_COUNTERPARTY RCP ON RC.HOLDING_ID = RCP.HOLDING_ID
    GROUP BY RCP.NAME
   )
   SELECT Counterparty_Name, Total_Collateral
   FROM CollateralCTE
   ORDER BY Total_Collateral DESC;

4. "Locate funds that have both securities lending activities and repurchase agreements."
   SQL: 
   WITH SecuritiesLending AS (
       SELECT ACCESSION_NUMBER
       FROM SECURITIES_LENDING
       WHERE IS_LOAN_BY_FUND = 'Y'
   ),
   RepurchaseAgreements AS (
       SELECT ACCESSION_NUMBER
       FROM REPURCHASE_AGREEMENT
   )
   SELECT F.SERIES_NAME
   FROM FUND_REPORTED_INFO F
   WHERE F.ACCESSION_NUMBER IN (SELECT ACCESSION_NUMBER FROM SecuritiesLending)
     AND F.ACCESSION_NUMBER IN (SELECT ACCESSION_NUMBER FROM RepurchaseAgreements);

5. "Find borrowers who have borrowed more than $5,000,000, including their names and LEIs."
   SQL: 
   WITH BorrowedAmounts AS (
       SELECT BORROWER_ID, SUM(AGGREGATE_VALUE) AS Total_Borrowed
       FROM BORROWER
       GROUP BY BORROWER_ID
       HAVING SUM(AGGREGATE_VALUE) > 5000000
   )
   SELECT B.NAME, B.LEI, BA.Total_Borrowed
   FROM BORROWER B
   JOIN BorrowedAmounts BA ON B.BORROWER_ID = BA.BORROWER_ID;

6. "List all derivative counterparties along with the number of derivative instruments they are involved in."
   SQL: 
   WITH CounterpartyCounts AS (
       SELECT DC.DERIVATIVE_COUNTERPARTY_NAME, COUNT(*) AS Instrument_Count
       FROM DERIVATIVE_COUNTERPARTY DC
       JOIN FUND_REPORTED_HOLDING H ON DC.HOLDING_ID = H.HOLDING_ID
       JOIN DEBT_SECURITY D ON H.HOLDING_ID = D.HOLDING_ID
       GROUP BY DC.DERIVATIVE_COUNTERPARTY_NAME
   )
   SELECT DERIVATIVE_COUNTERPARTY_NAME, Instrument_Count
   FROM CounterpartyCounts
   ORDER BY Instrument_Count DESC;

7. "Compute the average annualized rate for debt securities grouped by coupon type."
   SQL: 
   WITH RateAverages AS (
       SELECT DS.COUPON_TYPE, AVG(DS.ANNUALIZED_RATE) AS Avg_Annualized_Rate
       FROM DEBT_SECURITY DS
       WHERE DS.ANNUALIZED_RATE IS NOT NULL
       GROUP BY DS.COUPON_TYPE
   )
   SELECT COUPON_TYPE, Avg_Annualized_Rate
   FROM RateAverages
   ORDER BY Avg_Annualized_Rate DESC;

8. "Get funds that have experienced a net decrease in assets over the last three reporting periods."
   SQL: 
   WITH AssetChanges AS (
       SELECT F.ACCESSION_NUMBER, F.SERIES_NAME, S.REPORT_DATE, F.NET_ASSETS,
              LAG(F.NET_ASSETS, 1) OVER (PARTITION BY F.SERIES_NAME ORDER BY S.REPORT_DATE) AS Previous_Period_Assets
       FROM FUND_REPORTED_INFO F
       JOIN SUBMISSION S ON F.ACCESSION_NUMBER = S.ACCESSION_NUMBER
   )
   SELECT DISTINCT AC.SERIES_NAME
   FROM AssetChanges AC
   WHERE AC.NET_ASSETS < AC.Previous_Period_Assets
     AND AC.Previous_Period_Assets IS NOT NULL;

9. "Identify issuers with more than three different securities holdings, including their names and CUSIPs."
   SQL: 
   WITH IssuerHoldings AS (
       SELECT H.ISSUER_NAME, H.ISSUER_CUSIP, COUNT(DISTINCT H.HOLDING_ID) AS Holding_Count
       FROM FUND_REPORTED_HOLDING H
       GROUP BY H.ISSUER_NAME, H.ISSUER_CUSIP
       HAVING COUNT(DISTINCT H.HOLDING_ID) > 3
   )
   SELECT ISSUER_NAME, ISSUER_CUSIP, Holding_Count
   FROM IssuerHoldings
   ORDER BY Holding_Count DESC;

10. "Calculate the total notional amount of derivatives per currency and identify the top 3 currencies by notional amount."
    SQL: 
    WITH NotionalSums AS (
        SELECT ODNA.CURRENCY_CODE, SUM(ODNA.NOTIONAL_AMOUNT) AS Total_Notional
        FROM OTHER_DERIV_NOTIONAL_AMOUNT ODNA
        GROUP BY ODNA.CURRENCY_CODE
    )
    SELECT CURRENCY_CODE, Total_Notional
    FROM NotionalSums
    ORDER BY Total_Notional DESC
    LIMIT 3;

11. "List funds with liquidation preferences exceeding their net assets."
    SQL: 
    WITH FundPreferences AS (
        SELECT F.SERIES_NAME, F.LIQUIDATION_PREFERENCE, F.NET_ASSETS
        FROM FUND_REPORTED_INFO F
    )
    SELECT SERIES_NAME, LIQUIDATION_PREFERENCE, NET_ASSETS
    FROM FundPreferences
    WHERE LIQUIDATION_PREFERENCE > NET_ASSETS;

12. "Find all convertible securities that are contingent and have a conversion ratio above 1.5."
    SQL: 
    WITH ConvertibleCTE AS (
        SELECT DS.HOLDING_ID, CSC.CONVERSION_RATIO
        FROM DEBT_SECURITY DS
        JOIN CONVERTIBLE_SECURITY_CURRENCY CSC ON DS.HOLDING_ID = CSC.HOLDING_ID
        WHERE DS.IS_CONVTIBLE_CONTINGENT = 'Y' AND CSC.CONVERSION_RATIO > 1.5
    )
    SELECT HOLDING_ID, CONVERSION_RATIO
    FROM ConvertibleCTE;

13. "Analyze the distribution of asset categories within the top 10 largest funds by total assets."
    SQL: 
    WITH TopFunds AS (
        SELECT SERIES_NAME, ACCESSION_NUMBER
        FROM FUND_REPORTED_INFO
        ORDER BY TOTAL_ASSETS DESC
        LIMIT 10
    ),
    AssetDistribution AS (
        SELECT H.ASSET_CAT, COUNT(*) AS Category_Count
        FROM FUND_REPORTED_HOLDING H
        JOIN TopFunds T ON H.ACCESSION_NUMBER = T.ACCESSION_NUMBER
        GROUP BY H.ASSET_CAT
    )
    SELECT ASSET_CAT, Category_Count
    FROM AssetDistribution
    ORDER BY Category_Count DESC;
    
14. "Find the top 10 funds with the highest average monthly returns in the past quarter."
   SQL: 
   WITH AvgMonthlyReturn AS (
       SELECT ACCESSION_NUMBER, 
              (MONTHLY_TOTAL_RETURN1 + MONTHLY_TOTAL_RETURN2 + MONTHLY_TOTAL_RETURN3) / 3.0 AS Avg_Return
       FROM MONTHLY_TOTAL_RETURN
   )
   SELECT F.SERIES_NAME, A.ACCESSION_NUMBER, A.Avg_Return
   FROM AvgMonthlyReturn A
   JOIN FUND_REPORTED_INFO F ON A.ACCESSION_NUMBER = F.ACCESSION_NUMBER
   ORDER BY A.Avg_Return DESC
   LIMIT 10;

15. "Compare the latest net asset values of the top 5 performing funds."
    SQL: 
    WITH TopPerformingFunds AS (
        SELECT 
            ACCESSION_NUMBER, 
            (MONTHLY_TOTAL_RETURN1 + MONTHLY_TOTAL_RETURN2 + MONTHLY_TOTAL_RETURN3) / 3.0 AS Avg_Return
        FROM 
            MONTHLY_TOTAL_RETURN
        ORDER BY 
            Avg_Return DESC
        LIMIT 5
    )
    SELECT 
        FR.SERIES_NAME, 
        FR.NET_ASSETS, 
        TP.Avg_Return
    FROM 
        TopPerformingFunds TP
    JOIN 
        FUND_REPORTED_INFO FR ON TP.ACCESSION_NUMBER = FR.ACCESSION_NUMBER;

16. "Calculate the overall average return across all funds for the most recent month."
    SQL: 
    WITH LatestReturns AS (
        SELECT 
            M.ACCESSION_NUMBER, 
            M.MONTHLY_TOTAL_RETURN1
        FROM 
            MONTHLY_TOTAL_RETURN M
        JOIN 
            SUBMISSION S ON M.ACCESSION_NUMBER = S.ACCESSION_NUMBER
        WHERE 
            S.REPORT_DATE = (SELECT MAX(REPORT_DATE) FROM SUBMISSION)
    )
    SELECT 
        AVG(MONTHLY_TOTAL_RETURN1) AS Average_Return
    FROM 
        LatestReturns;

17. "Find the interest rate risk for each fund and identify those with the highest risk scores."
    SQL: 
    WITH InterestRiskScores AS (
        SELECT 
            IR.ACCESSION_NUMBER, 
            -- Calculating composite risk score by summing absolute values of DV01 and DV100 columns
            (ABS(CAST(IR.INTRST_RATE_CHANGE_3MON_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_1YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_5YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_10YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_30YR_DV01 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_3MON_DV100 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_1YR_DV100 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_5YR_DV100 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_10YR_DV100 AS FLOAT)) +
            ABS(CAST(IR.INTRST_RATE_CHANGE_30YR_DV100 AS FLOAT))
            ) AS Composite_Risk_Score
        FROM 
            INTEREST_RATE_RISK IR
    )
    SELECT 
        FR.SERIES_NAME, 
        FR.ACCESSION_NUMBER, 
        IRS.Composite_Risk_Score
    FROM 
        InterestRiskScores IRS
    JOIN 
        FUND_REPORTED_INFO FR ON IRS.ACCESSION_NUMBER = FR.ACCESSION_NUMBER
    ORDER BY 
        IRS.Composite_Risk_Score DESC
    LIMIT 5;

18. "Analyze the composition of fund portfolios by categorizing assets and their total values."
    SQL: 
    WITH PortfolioComposition AS (
    SELECT 
        ACCESSION_NUMBER, 
        ASSET_CAT, 
        SUM(CAST(CURRENCY_VALUE AS FLOAT)) AS Total_Value
    FROM 
        FUND_REPORTED_HOLDING
    GROUP BY 
        ACCESSION_NUMBER, 
        ASSET_CAT
    )
    SELECT 
        F.SERIES_NAME, 
        PC.ASSET_CAT, 
        PC.Total_Value
    FROM 
        PortfolioComposition PC
    JOIN 
        FUND_REPORTED_INFO F ON PC.ACCESSION_NUMBER = F.ACCESSION_NUMBER
    ORDER BY 
        F.SERIES_NAME, 
        PC.Total_Value DESC;

19. "Identify the most common asset categories across all fund portfolios."
    SQL: 
    WITH AssetCounts AS (
        SELECT ASSET_CAT, COUNT(*) AS Count
        FROM FUND_REPORTED_HOLDING
        GROUP BY ASSET_CAT
    )
    SELECT ASSET_CAT, Count
    FROM AssetCounts
    ORDER BY Count DESC
    LIMIT 5;

20. "Retrieve funds that have experienced a net decrease in assets over the last three reporting periods."
   SQL: 
   WITH AssetChanges AS (
       SELECT F.ACCESSION_NUMBER, F.SERIES_NAME, S.REPORT_DATE, F.NET_ASSETS,
              LAG(F.NET_ASSETS, 1) OVER (PARTITION BY F.SERIES_NAME ORDER BY S.REPORT_DATE) AS Previous_Period_Assets
       FROM FUND_REPORTED_Iimport os
print(os.getcwd())NFO F
       JOIN SUBMISSION S ON F.ACCESSION_NUMBER = S.ACCESSION_NUMBER
   )
   SELECT DISTINCT AC.SERIES_NAME
   FROM AssetChanges AC
   WHERE AC.NET_ASSETS < AC.Previous_Period_Assets
     AND AC.Previous_Period_Assets IS NOT NULL;"""



In [ ]:
pattern = r'"\s*(.*?)\s*"\s*SQL:\s*(WITH.*?;)(?=\n\s*\d+|$)'
matches = re.findall(pattern, text, re.DOTALL)


In [ ]:
ground_truth_query = [match[1] for match in matches]
llm_query = [match[0] for match in matches]
print("Queries:", ground_truth_query)
print("SQL Statements:", llm_query)

In [ ]:
results = []
while len(results) < 20:
    results.append(None)
print(len(results))
print(len(llm_query))

print("Are the following queries the same?")
i=0
print("i = ", i)


In [ ]:
if i<len(llm_query):
    results[i]=(str(i)+'. '+ str(compare_csv.compare_csv_din(ground_truth_query[i],llm_query[i])))
i+=1 #1
print("i = ", i)


In [41]:
class PSLsh:
    def __init__(self, vectors, n_planes=10, n_tables=5, seed: int = 42):
        self.n_planes = n_planes
        self.n_tables = n_tables
        self.hash_tables = [{} for _ in range(n_tables)]
        self.random_planes = []
        
        np.random.seed(seed)
        
        for _ in range(n_tables):
            planes = np.random.randn(vectors.shape[1], n_planes)
            self.random_planes.append(planes)
            
        self.num_vectors = vectors.shape[0]
        self.vectors = vectors
        self.build_hash_tables()

    def build_hash_tables(self):
        for idx in range(self.num_vectors):
            vector = self.vectors[idx].toarray()[0]
            hashes = self.hash_vector(vector)
            for i, h in enumerate(hashes):
                if h not in self.hash_tables[i]:
                    self.hash_tables[i][h] = []
                self.hash_tables[i][h].append(idx)

    def hash_vector(self, vector):
        hashes = []
        for planes in self.random_planes:
            projections = np.dot(vector, planes)
            hash_code = ''.join(['1' if x > 0 else '0' for x in projections])
            hashes.append(hash_code)
        return hashes

    def query(self, vector):
        hashes = self.hash_vector(vector)
        candidates = set()
        for i, h in enumerate(hashes):
            candidates.update(self.hash_tables[i].get(h, []))
        return candidates


class ValueRetrieval:
    financial_terms = {
            'total': ['total', 'sum', 'aggregate', 'combined'],
            'assets': ['asset', 'holdings', 'investments', 'securities'],
            'liabilities': ['liability', 'debt', 'obligations'],
            'net': ['net', 'pure', 'adjusted'],
            'fund': ['fund', 'portfolio', 'investment vehicle'],
            'return': ['return', 'yield', 'profit', 'gain'],
            'monthly': ['monthly', 'month', 'monthly basis'],
            'rate': ['rate', 'percentage', 'ratio'],
            'risk': ['risk', 'exposure', 'vulnerability']
        }
    def __init__(self, schema_path: str = 'chatgpt_api/schema.json', lsh_seed: int = 42):
        load_dotenv()
        self.client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

        # Load schema
        print("DEBUG: Loading schema file:", schema_path)
        with open(schema_path, 'r') as f:
            self.schema = json.load(f)

        # Initialize lemmatizer and stop words
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
        # Build column name index
        self.column_index = self._build_column_index()
        
        # Build common financial terms dictionary
        

        # Build vectorizer and LSH for backup matching
        self.build_vectorizer_and_lsh(seed=lsh_seed)
        
        # Get schema relationships
        self.primary_keys, self.foreign_keys = self.discover_schema_relationships()

    def _build_column_index(self) -> Dict:
        """Build an index of all columns with their metadata."""
        column_index = {}
        tables = self.schema.get('schema', {}).get('tables', [])
        
        for table in tables:
            table_name = table.get('name', '').lower()
            for column in table.get('columns', []):
                column_name = column.get('name', '').lower()
                
                # Store both the full qualified name and column properties
                qualified_name = f"{table_name}.{column_name}"
                column_index[qualified_name] = {
                    'table': table_name,
                    'column': column_name,
                    'type': column.get('type', ''),
                    'words': self._split_column_name(column_name),
                    'synonyms': self._get_column_synonyms(column_name)
                }
                
        return column_index

    def _split_column_name(self, column_name: str) -> List[str]:
        """Split column name into individual words."""
        # Handle both underscore and camel case
        words = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', column_name)).split()
        words.extend(column_name.split('_'))
        return [word.lower() for word in words if word]

    def _get_column_synonyms(self, column_name: str) -> List[str]:
        """Get synonyms for words in column name."""
        words = self._split_column_name(column_name)
        synonyms = []
        
        for word in words:
            if word in self.financial_terms:
                synonyms.extend(self.financial_terms[word])
                
        return list(set(synonyms))

    def build_vectorizer_and_lsh(self, seed: int):
        self.vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 3), min_df=1, max_df=0.95)
        self.term_list = self.get_schema_terms()
        self.term_vectors = self.vectorizer.fit_transform(self.term_list)
        self.lsh = PSLsh(self.term_vectors, n_planes=10, n_tables=5)

    def get_schema_terms(self) -> List[str]:
        terms = []
        tables = self.schema.get('schema', {}).get('tables', [])
        for table in tables:
            table_name = table.get('name', '').lower()
            terms.append(table_name)
            for column in table.get('columns', []):
                column_name = column.get('name', '').lower()
                terms.append(f"{table_name}.{column_name}")
        return terms

    def discover_schema_relationships(self):
        # Define our primary keys and foreign keys here
        primary_keys = {
            'SUBMISSION': ['ACCESSION_NUMBER'],
            'REGISTRANT': ['ACCESSION_NUMBER'],
            'FUND_REPORTED_INFO': ['ACCESSION_NUMBER'],
            'INTEREST_RATE_RISK': ['ACCESSION_NUMBER', 'INTEREST_RATE_RISK_ID'],
            'BORROWER': ['ACCESSION_NUMBER', 'BORROWER_ID'],
            'BORROW_AGGREGATE': ['ACCESSION_NUMBER', 'BORROW_AGGREGATE_ID'],
            'MONTHLY_TOTAL_RETURN': ['ACCESSION_NUMBER', 'MONTHLY_TOTAL_RETURN_ID'],
            'MONTHLY_RETURN_CAT_INSTRUMENT': ['ACCESSION_NUMBER', 'ASSET_CAT', 'INSTRUMENT_KIND'],
            'FUND_VAR_INFO': ['ACCESSION_NUMBER'],
            'FUND_REPORTED_HOLDING': ['ACCESSION_NUMBER', 'HOLDING_ID'],
            'IDENTIFIERS': ['HOLDING_ID', 'IDENTIFIERS_ID'],
            'DEBT_SECURITY': [],  
            'DEBT_SECURITY_REF_INSTRUMENT': ['HOLDING_ID', 'DEBT_SECURITY_REF_ID'],
            'CONVERTIBLE_SECURITY_CURRENCY': ['HOLDING_ID', 'CONVERTIBLE_SECURITY_ID'],
            'REPURCHASE_AGREEMENT': ['HOLDING_ID'],
            'REPURCHASE_COUNTERPARTY': ['HOLDING_ID', 'REPURCHASE_COUNTERPARTY_ID'],
            'REPURCHASE_COLLATERAL': ['HOLDING_ID', 'REPURCHASE_COLLATERAL_ID'],
            'DERIVATIVE_COUNTERPARTY': ['HOLDING_ID', 'DERIVATIVE_COUNTERPARTY_ID'],
            'SWAPTION_OPTION_WARNT_DERIV': ['HOLDING_ID'],
            'DESC_REF_INDEX_BASKET': ['HOLDING_ID'],
            'DESC_REF_INDEX_COMPONENT': ['HOLDING_ID', 'DESC_REF_INDEX_COMPONENT_ID'],
            'DESC_REF_OTHER': ['HOLDING_ID', 'DESC_REF_OTHER_ID'],
            'FUT_FWD_NONFOREIGNCUR_CONTRACT': ['HOLDING_ID'],
            'FWD_FOREIGNCUR_CONTRACT_SWAP': ['HOLDING_ID'],
            'NONFOREIGN_EXCHANGE_SWAP': ['HOLDING_ID'],
            'FLOATING_RATE_RESET_TENOR': ['HOLDING_ID', 'RATE_RESET_TENOR_ID'],
            'OTHER_DERIV': ['HOLDING_ID'],
            'OTHER_DERIV_NOTIONAL_AMOUNT': ['HOLDING_ID', 'OTHER_DERIV_NOTIONAL_AMOUNT_ID'],
            'SECURITIES_LENDING': ['HOLDING_ID'],
            'EXPLANATORY_NOTE': ['ACCESSION_NUMBER', 'EXPLANATORY_NOTE_ID']
        }

        foreign_keys = [
            # ACCESSION_NUMBER relationships
            'REGISTRANT.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'INTEREST_RATE_RISK.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'BORROWER.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'BORROW_AGGREGATE.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'MONTHLY_TOTAL_RETURN.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'MONTHLY_RETURN_CAT_INSTRUMENT.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'FUND_VAR_INFO.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'FUND_REPORTED_HOLDING.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'EXPLANATORY_NOTE.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',
            'SUBMISSION.ACCESSION_NUMBER = FUND_REPORTED_INFO.ACCESSION_NUMBER',

            # HOLDING_ID relationships
            'IDENTIFIERS.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DEBT_SECURITY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DEBT_SECURITY_REF_INSTRUMENT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'CONVERTIBLE_SECURITY_CURRENCY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'REPURCHASE_AGREEMENT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'REPURCHASE_COUNTERPARTY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'REPURCHASE_COLLATERAL.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DERIVATIVE_COUNTERPARTY.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'SWAPTION_OPTION_WARNT_DERIV.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DESC_REF_INDEX_BASKET.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DESC_REF_INDEX_COMPONENT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'DESC_REF_OTHER.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'FUT_FWD_NONFOREIGNCUR_CONTRACT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'FWD_FOREIGNCUR_CONTRACT_SWAP.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'NONFOREIGN_EXCHANGE_SWAP.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'FLOATING_RATE_RESET_TENOR.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'OTHER_DERIV.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'OTHER_DERIV_NOTIONAL_AMOUNT.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID',
            'SECURITIES_LENDING.HOLDING_ID = FUND_REPORTED_HOLDING.HOLDING_ID'
        ]

        formatted_pks = []
        for table, keys in primary_keys.items():
            for key in keys:
                formatted_pks.append(f"{table}.{key}")

        return formatted_pks, foreign_keys

    def find_similar_words(self, word: str) -> List[Tuple[str, float]]:
        """Enhanced matching using multiple techniques."""
        if not word:
            return []

        word = word.lower()
        print(f"\nDEBUG: Finding matches for '{word}'")
        
        matches = []
        
        # 1. Direct matching with column names and their components
        for qualified_name, metadata in self.column_index.items():
            score = 0.0
            
            # Check exact matches in column words
            if word in metadata['words']:
                matches.append((qualified_name, 1.0))
                continue
                
            # Check synonyms
            if word in self.financial_terms.get(word, []):
                matches.append((qualified_name, 0.9))
                continue
            
            # Fuzzy match with column words
            for col_word in metadata['words']:
                ratio = fuzz.ratio(word, col_word) / 100.0
                if ratio > score:
                    score = ratio
            
            # Fuzzy match with synonyms
            for term, synonyms in self.financial_terms.items():
                if term in metadata['words']:
                    for synonym in synonyms:
                        ratio = fuzz.ratio(word, synonym) / 100.0
                        if ratio > score:
                            score = ratio * 0.9  # Slightly lower weight for synonym matches
            
            if score > 0.6:  # Only include if similarity is above 60%
                matches.append((qualified_name, score))

        # 2. LSH-based matching as backup
        if len(matches) < 5:  # If we have fewer than 5 matches, try LSH
            try:
                word_vector = self.vectorizer.transform([word]).toarray()[0]
                candidate_indices = self.lsh.query(word_vector)
                
                for idx in candidate_indices:
                    term = self.term_list[idx]
                    if not any(term == m[0] for m in matches):  # Avoid duplicates
                        candidate_vector = self.term_vectors[idx].toarray()[0]
                        dist = np.linalg.norm(word_vector - candidate_vector)
                        sim = 1 / (1 + dist)
                        if sim > 0.5:  # Only include if similarity is above 50%
                            matches.append((term, sim * 0.8))
            except Exception as e:
                print(f"LSH matching failed: {e}")

        # Remove duplicates keeping highest score and sort by score
        unique_matches = {}
        for term, score in matches:
            if term not in unique_matches or score > unique_matches[term]:
                unique_matches[term] = score
        
        matches = [(term, score) for term, score in unique_matches.items()]
        matches.sort(key=lambda x: x[1], reverse=True)
        
        # Print debug info
        print(f"Found {len(matches)} matches for '{word}':")
        for match, score in matches[:5]:
            print(f"  {match}: {score:.4f}")
        
        return matches[:5] if matches else [('fund_reported_info.total_assets', 0.6)] if word in ['total', 'asset', 'assets'] else []

    def extract_keywords(self, question: str) -> Dict:
        system_prompt = """You are an expert financial data analyst specializing in natural language understanding and database schema analysis.
Your task is to analyze questions about financial data and extract key components that will help in database queries.

Objective: Break down the given question into essential components that will help formulate a database query.

Instructions:
1. Read the question carefully to identify:
- Individual keywords that map to database columns or values
- Technical terms related to financial data
- Named entities (companies, funds, locations)
- Numerical thresholds or values

2. For each identified element, categorize it as:
- keywords: Individual significant words that might match database columns
- keyphrases: Multi-word expressions that represent single concepts
- named_entities: Specific names of companies, funds, or locations
- numerical_values: Any numbers, amounts, or thresholds

3. Return a JSON object with these categories."""

        few_shot_examples = """
Example Question: "Which PIMCO funds were registered between 2020 and 2023 with California addresses?"
{
    "keywords": ["funds", "registered", "addresses"],
    "keyphrases": ["PIMCO funds"],
    "named_entities": ["PIMCO", "California"],
    "numerical_values": ["2020", "2023"]
}

Example Question: "Show me BlackRock funds with total assets over 1 billion managed in New York"
{
    "keywords": ["funds", "assets", "managed"],
    "keyphrases": ["total assets"],
    "named_entities": ["BlackRock", "New York"],
    "numerical_values": ["1 billion"]
}"""

        formatted_prompt = system_prompt
        user_prompt = f"Question: \"{question}\"\n\nExtract the key components and return as JSON."

        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": formatted_prompt},
                {"role": "user", "content": few_shot_examples + "\n" + user_prompt}
            ],
            response_format={"type": "json_object"}
        )

        return json.loads(response.choices[0].message.content)

    def preprocess_text(self, text: str) -> List[str]:
        """Tokenize and lemmatize input text, removing stop words."""
        tokens = nltk.word_tokenize(text.lower())
        filtered_tokens = [word for word in tokens if word not in self.stop_words and word.isalnum()]
        lemmatized_tokens = [self.lemmatizer.lemmatize(token) for token in filtered_tokens]
        return lemmatized_tokens
    
    def process_question(self, question: str) -> Dict:
        # Extract keywords using gpt
        extracted_info = self.extract_keywords(question)

        words = []
        for key in ['keywords', 'keyphrases', 'named_entities', 'numerical_values']:
            words.extend(extracted_info.get(key, []))

        # Preprocess the words (lemmatize, remove stop words)
        processed_words = []
        for word in words:
            processed_words.extend(self.preprocess_text(word))

        # Remove duplicates
        processed_words = list(set(processed_words))

        # Find similar columns for each word
        similar_matches = {}
        for word in processed_words:
            similar_matches[word] = self.find_similar_words(word)

        # Combine the results
        result = {
            "question": question,
            "extracted_info": extracted_info,
            "processed_words": processed_words,
            "similar_matches": similar_matches,
            "schema_relationships": {
                "primary_keys": self.primary_keys,
                "foreign_keys": self.foreign_keys
            }
        }
        return result
    


if __name__ == "__main__":
    vr = ValueRetrieval(schema_path='chatgpt_api/schema.json')

    question = "Show me all funds with total asset over 1 billion"
    result = vr.process_question(question)

    from pprint import pprint
    pprint(result)

DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/virounikamina/Desktop/PIMCO-Text2SQL/env/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are an expert financial data analyst specializing in natural language understanding and database schema analysis.\nYour task is to analyze questions about financial data and extract key components that will help in database queries.\n\nObjective: Break down the given question into essential components that will help formulate a database query.\n\nInstructions:\n1. Read the question carefully to identify:\n- Individual keywords that map to database columns or values\n- Technical terms related to financial data\n- Named entities (c

DEBUG: Loading schema file: chatgpt_api/schema.json


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 20 Nov 2024 21:43:11 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'781'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999576'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'12ms'), (b'x-request-id', b'req_207352607ee62fbe93be52f59fb2c7a0'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=LnFqub9I4MYwyU1K0aXt7x9SRvjHDA58Wf0ukGNPUj8-1732138991-1.0.1.1-efTD1Pze4.Emk4DFVF0j9kDyPTV.OsZZfrRcy6JKjse


DEBUG: Finding matches for 'fund'
Found 279 matches for 'fund':
  securities_lending.is_loan_by_fund: 1.0000
  registrant.accession_number: 0.9000
  registrant.cik: 0.9000
  registrant.registrant_name: 0.9000
  registrant.file_num: 0.9000

DEBUG: Finding matches for 'asset'
Found 7 matches for 'asset':
  fund_reported_holding.asset_cat: 1.0000
  monthly_return_cat_instrument.asset_cat: 1.0000
  fund_reported_info.total_assets: 0.9000
  fund_reported_info.net_assets: 0.9000
  fund_reported_info.assets_attrbt_to_misc_security: 0.9000

DEBUG: Finding matches for '1'
Found 0 matches for '1':

DEBUG: Finding matches for 'billion'
Found 2 matches for 'billion':
  submission.filing_date: 0.6154
  submission.is_last_filing: 0.6154

DEBUG: Finding matches for 'total'
Found 279 matches for 'total':
  fund_reported_info.total_assets: 1.0000
  fund_reported_info.total_liabilities: 1.0000
  monthly_total_return.monthly_total_return_id: 1.0000
  monthly_total_return.monthly_total_return1: 1.0000
  

In [43]:
############################################ CLASSIFICATION
classification_prompt = '''Q: "Find the filing date and submission number of all reports filed for an NPORT-P submission."
schema_links: [submission.filing_date, submission.sub_type = "NPORT-P", submission.accession_number]
A: Let’s think step by step. The SQL query for the question "Find the filing date and submission number of all reports filed for an NPORT-P submission." needs these tables = [submission], so we don't need JOIN.
Plus, it doesn't require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""]. 
So, we don't need JOIN and don't need nested queries, then the SQL query can be classified as "EASY".
Label: "EASY"

Q: "Get the names and CIK of registrants who are located in California."
schema_links: [registrant.registrant_name, registrant.cik, registrant.state = "US-CA"]
A: Let’s think step by step. The SQL query for the question "Get the names and CIK of registrants who are located in California." needs these tables = [registrant], so we don't need JOIN.
Plus, it doesn't require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""]. 
So, we don't need JOIN and don't need nested queries, then the SQL query can be classified as "EASY".
Label: "EASY"

Q: "Find the names and CIK of registrants in California, but only for those whose total assets are above 100 million."
schema_links: [registrant.registrant_name, registrant.cik, registrant.state = "US-CA", fund_reported_info.total_assets > 100000000]
A: Let's analyze this. The query involves data from two tables: "registrant" for registrant details and "fund_reported_info" for total assets. Since we need to check if total assets exceed 100 million, a nested query is necessary to filter based on this condition. This is a nested query. So, the SQL query can be classified as "NESTED."
Label: "NESTED"

'''

def classification_prompt_maker(question,relevant_schema_links):
  instruction = "# For the given question, classify it as EASY, NON-NESTED, or NESTED based on nested queries and JOIN.\n"
  instruction += "\nif need nested queries: predict NESTED\n"
  instruction += "elif need JOIN and don't need nested queries: predict NON-NESTED\n"
  instruction += "elif don't need JOIN and don't need nested queries: predict EASY\n\n"
  prompt = instruction + classification_prompt + 'Q: "' + question + '\nrelevant_schema_links: ' + relevant_schema_links + '\nA: Let’s think step by step.'
  return prompt

def process_question_classification(question, relevant_schema_links):
    classification = None
    attempts = 0
    while classification is None and attempts < 3:
        try:
            print("Attempting classification...")
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[{
                    "role": "user", 
                    "content": classification_prompt_maker(question, relevant_schema_links=relevant_schema_links)
                }],
                n=1,
                stream=False,
                temperature=0.0,
                max_tokens=300,  # Reduced max tokens
                top_p=1.0,
                frequency_penalty=0.0,
                presence_penalty=0.0
                # Removed the stop sequence
            )
            classification = response.choices[0].message.content
            print("Raw response:", classification)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            time.sleep(3)
            attempts += 1
    
    if classification is None:
        print("Failed to get classification after 3 attempts")
        return '"NESTED"'
        
    try:
        predicted_class = classification.split("Label: ")[1].strip().strip('"')
        return f'"{predicted_class}"'
    except:
        print("Slicing error for the classification module")
        return '"NESTED"'


############################################ SQL GENERATION
easy_prompt = '''Q: "Find the issuers with a balance greater than 1 million."
Schema_links: [fund_reported_holding.balance]
SQL: SELECT DISTINCT issuer_name 
      FROM fund_reported_holding 
      WHERE balance > 1000000
'''

medium_prompt = '''Q: "Find the total upfront payments and receipts for swaps with fixed rate receipts."
Schema_links: [nonforeign_exchange_swap.upfront_payment, nonforeign_exchange_swap.upfront_receipt, nonforeign_exchange_swap.fixed_rate_receipt]
A: Let’s think step by step. For creating the SQL for the given question, we need to filter the swaps that have fixed rate receipts. Then, sum up the upfront payments and receipts. First, create an intermediate representation, then use it to construct the SQL query.
Intermediate_representation: 
SELECT SUM(nonforeign_exchange_swap.upfront_payment) + SUM(nonforeign_exchange_swap.upfront_receipt) 
FROM nonforeign_exchange_swap 
WHERE nonforeign_exchange_swap.fixed_rate_receipt IS NOT NULL
SQL: 
SELECT SUM(upfront_payment) + SUM(upfront_receipt) 
FROM nonforeign_exchange_swap 
WHERE fixed_rate_receipt IS NOT NULL
'''

hard_prompt = '''Q: "Find the borrowers with aggregate value greater than $1 million and whose interest rate change at 10-year maturity for a 100 basis point change is positive."
Schema_links: [borrower.aggregate_value, borrower.name, interest_rate_risk.intrst_rate_change_10yr_dv100]
A: Let's think step by step. First, we need to filter borrowers with aggregate values greater than $1 million. Then, we need to check for interest rate changes at 10-year maturity where the change is positive. 
The SQL query for the sub-question "What are the borrowers with aggregate value greater than $1 million and positive interest rate change at 10-year maturity for 100 basis points?" is:

Intermediate_representation: 
SELECT borrower.name 
FROM borrower 
JOIN interest_rate_risk 
ON borrower.accession_number = interest_rate_risk.accession_number 
WHERE borrower.aggregate_value > 1000000 
AND interest_rate_risk.intrst_rate_change_10yr_dv100 > 0

SQL: 
SELECT borrower.name 
FROM borrower 
JOIN interest_rate_risk 
ON borrower.accession_number = interest_rate_risk.accession_number 
WHERE borrower.aggregate_value > 1000000 
AND interest_rate_risk.intrst_rate_change_10yr_dv100 > 0
'''

def hard_prompt_maker(question,database,schema_links,sub_questions=""):
    instruction = "# Use the intermediate representation and the schema links to generate the SQL queries for each of the questions.\n"
    if sub_questions=="":
        stepping = f'''\nA: Let's think step by step. "{question}" can be solved by first solving a sub-question using nested queries".'''
    else:
        stepping = f'''\nA: Let's think step by step. "{question}" can be solved by first solving the answer to the following sub-question "{sub_questions}".'''
    prompt = instruction + hard_prompt+ chat_prompt.gpt_queries_hard+ 'Q: "' + question + '"' + '\nschema_links: ' + schema_links + stepping +'\nThe SQL query for the sub-question:"'
    return prompt

def medium_prompt_maker(question,database,schema_links):
    instruction = "# Use the the schema links and Intermediate_representation to generate the SQL queries for each of the questions.\n"
    prompt = instruction + medium_prompt + chat_prompt.gpt_queries_medium+ 'Q: "' + question + '\nSchema_links: ' + schema_links + '\nA: Let’s think step by step.'
    return prompt

def easy_prompt_maker(question,database,schema_links):
    instruction = "# Use the the schema links to generate the SQL queries for each of the questions.\n"
    prompt = instruction + easy_prompt + chat_prompt.gpt_queries_easy + 'Q: "' + question + '\nSchema_links: ' + schema_links + '\nSQL:'
    return prompt


In [44]:
# Test classification directly
test_question = "Show me all funds with total assets over 1 billion"
test_schema_links = "[fund_reported_info.total_assets > 1000000000]"

# Generate classification prompt
prompt = classification_prompt_maker(test_question, test_schema_links)
print("Classification Prompt:")
print(prompt)

# Get classification
classification = process_question_classification(test_question, test_schema_links)
print("\nClassification Result:")
print(classification)


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': '# For the given question, classify it as EASY, NON-NESTED, or NESTED based on nested queries and JOIN.\n\nif need nested queries: predict NESTED\nelif need JOIN and don\'t need nested queries: predict NON-NESTED\nelif don\'t need JOIN and don\'t need nested queries: predict EASY\n\nQ: "Find the filing date and submission number of all reports filed for an NPORT-P submission."\nschema_links: [submission.filing_date, submission.sub_type = "NPORT-P", submission.accession_number]\nA: Let’s think step by step. The SQL query for the question "Find the filing date and submission number of all reports filed for an NPORT-P submission." needs these tables = [submission], so we don\'t need JOIN.\nPlus, it doesn\'t require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""]. \nSo, we d

Classification Prompt:
# For the given question, classify it as EASY, NON-NESTED, or NESTED based on nested queries and JOIN.

if need nested queries: predict NESTED
elif need JOIN and don't need nested queries: predict NON-NESTED
elif don't need JOIN and don't need nested queries: predict EASY

Q: "Find the filing date and submission number of all reports filed for an NPORT-P submission."
schema_links: [submission.filing_date, submission.sub_type = "NPORT-P", submission.accession_number]
A: Let’s think step by step. The SQL query for the question "Find the filing date and submission number of all reports filed for an NPORT-P submission." needs these tables = [submission], so we don't need JOIN.
Plus, it doesn't require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""]. 
So, we don't need JOIN and don't need nested queries, then the SQL query can be classified as "EASY".
Label: "EASY"

Q: "Get the names and CIK of registrants who

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 20 Nov 2024 21:43:50 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'4456'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'300000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'299124'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'175ms'), (b'x-request-id', b'req_856ba4a7c9db16a8228b22b117843b6f'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=Qs2WG3DjpcT5UK.vHEhKkyOWRwFARjvTMpOtiEdPa94-1732139030-1.0.1.1-XrRkyOA_Cfkqb7ZHuan_t5V7EdZDaRJFRaJrHyE6NLH

Raw response: The SQL query for the question "Show me all funds with total assets over 1 billion" needs these tables = [fund_reported_info], so we don't need JOIN.
Plus, it doesn't require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""]. 
So, we don't need JOIN and don't need nested queries, then the SQL query can be classified as "EASY".
Label: "EASY"

Classification Result:
"EASY"


In [46]:
def get_schema_links_from_retrieval(value_retrieval, question):
    retrieval_result = value_retrieval.process_question(question)
    
    # Get the most relevant schema links only
    schema_links = []
    
    # 1. Get the main table/column matches
    for word, matches in retrieval_result['similar_matches'].items():
        if matches:
            # Only take the top match if score > 0.7
            top_match = matches[0]  # (match, score)
            if top_match[1] > 0.7:
                # Handle numerical values
                if word in retrieval_result['extracted_info'].get('numerical_values', []):
                    if 'billion' in word.lower():
                        schema_links.append(f"{top_match[0]} > 1000000000")
                    elif 'million' in word.lower():
                        schema_links.append(f"{top_match[0]} > 1000000")
                    else:
                        schema_links.append(f"{top_match[0]} > {word}")
                else:
                    schema_links.append(top_match[0])
    
    # 2. Add only the foreign keys that connect tables we found
    if len(schema_links) > 1:
        tables_needed = set()
        for link in schema_links:
            if '.' in link:
                tables_needed.add(link.split('.')[0].upper())
        
        # Only add foreign keys that connect our needed tables
        for fk in retrieval_result['schema_relationships']['foreign_keys']:
            tables_in_fk = set(part.split('.')[0] for part in fk.split(' = '))
            if tables_in_fk.intersection(tables_needed):
                schema_links.append(fk)
    
    print("Filtered schema links:", schema_links)
    return str(schema_links)

# Test it
vr = ValueRetrieval(schema_path='chatgpt_api/schema.json')
question = "Show me all funds with total assets over 1 billion"

schema_links = get_schema_links_from_retrieval(vr, question)
print("\nTesting classification with filtered schema links...")
classification = process_question_classification(question, schema_links)
print("Classification:", classification)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/virounikamina/Desktop/PIMCO-Text2SQL/env/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are an expert financial data analyst specializing in natural language understanding and database schema analysis.\nYour task is to analyze questions about financial data and extract key components that will help in database queries.\n\nObjective: Break down the given question into essential components that will help formulate a database query.\n\nInstructions:\n1. Read the question carefully to identify:\n- Individual keywords that map to database columns or values\n- Technical terms related to financial data\n- Named entities (companies, funds, locations)\n- Numerical thresholds or values\n\n2. For each iden

DEBUG: Loading schema file: chatgpt_api/schema.json


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 20 Nov 2024 21:45:32 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'742'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999576'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'12ms'), (b'x-request-id', b'req_aee87e512475399559d01b7cd3689cb7'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=8lRpEkaHDlU77PcceZgrxRr4T5TY22Ox.6P5avQ.DnM-1732139132-1.0.1.1-9hF70tVhqHuxvuGFEmqDokb2XMBkl_SwG07ovNu5Vjt


DEBUG: Finding matches for 'fund'
Found 279 matches for 'fund':
  securities_lending.is_loan_by_fund: 1.0000
  registrant.accession_number: 0.9000
  registrant.cik: 0.9000
  registrant.registrant_name: 0.9000
  registrant.file_num: 0.9000

DEBUG: Finding matches for 'asset'
Found 7 matches for 'asset':
  fund_reported_holding.asset_cat: 1.0000
  monthly_return_cat_instrument.asset_cat: 1.0000
  fund_reported_info.total_assets: 0.9000
  fund_reported_info.net_assets: 0.9000
  fund_reported_info.assets_attrbt_to_misc_security: 0.9000

DEBUG: Finding matches for '1'
Found 0 matches for '1':

DEBUG: Finding matches for 'billion'
Found 2 matches for 'billion':
  submission.filing_date: 0.6154
  submission.is_last_filing: 0.6154

DEBUG: Finding matches for 'total'
Found 279 matches for 'total':
  fund_reported_info.total_assets: 1.0000
  fund_reported_info.total_liabilities: 1.0000
  monthly_total_return.monthly_total_return_id: 1.0000
  monthly_total_return.monthly_total_return1: 1.0000
  

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 20 Nov 2024 21:45:38 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-annfuni26pdtuawdwdj6zorw'), (b'openai-processing-ms', b'5616'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'300000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'298579'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'284ms'), (b'x-request-id', b'req_d12ecaef606352ff310f0d35d62844d7'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8e5ba0aa8a902ab4-LAX'), (b'Content-Encoding', b'g

Raw response: The SQL query for the question "Show me all funds with total assets over 1 billion" needs these tables = [fund_reported_info], so we don't need JOIN.
Plus, it doesn't require nested queries with (INTERSECT, UNION, EXCEPT, IN, NOT IN), and we need the answer to the questions = [""]. 
So, we don't need JOIN and don't need nested queries, then the SQL query can be classified as "EASY".
Label: "EASY"
Classification: "EASY"
